# PedBE

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'pedbe'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2019
approved_by_author = '⌛'
citation = "McEwen, Lisa M., et al. \"The PedBE clock accurately estimates DNA methylation age in pediatric buccal cells.\" Proceedings of the National Academy of Sciences 117.38 (2020): 23329-23335."
doi = "https://doi.org/10.1073/pnas.1820843116"
notes = None

## Download necessary data

#### Download GitHub repository

In [3]:
github_url = "https://github.com/kobor-lab/Public-Scripts"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load data

#### From CSV file

In [4]:
df = pd.read_csv('Public-Scripts/datcoefInteresting94.csv')

## Extract features and weights

First, let's extract the features and weights:

In [5]:
df['feature'] = df['ID']
df['coefficient'] = df['Coef']

df.head()

,ID,Coef,corAgeTraining,corAgeTest,feature,coefficient
0,(Intercept),-2.097349,NaN,NaN,(Intercept),-2.097349
1,cg00059225,0.021960,0.866104,0.890372,cg00059225,0.021960
2,cg00085493,-0.100396,-0.780114,-0.809016,cg00085493,-0.100396
3,cg00095976,0.007872,0.687475,0.757393,cg00095976,0.007872
4,cg00609333,0.022824,0.629197,0.798287,cg00609333,0.022824


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [6]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

## Load weights into pyaging model

#### Linear model

In [7]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=94, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [8]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [9]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [10]:
def postprocessing_function(x):
    """
    Applies an anti-logarithmic linear transformation to a PyTorch tensor.
    """
    adult_age=20
    
    if x < 0:
        # Exponential transformation for negative values
        age_tensor = (1 + adult_age) * math.exp(x) - 1
    else:
        # Linear transformation for non-negative values
        age_tensor = (1 + adult_age) * x + adult_age
    return age_tensor
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = None

postprocessing = {
    'name': 'anti_log_linear',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Write clock dictionary

Let's put everything together and save:

In [11]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [12]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: Public-Scripts
Deleted folder: .ipynb_checkpoints
